In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F


In [2]:
data_dir = '../../../../BottleStoodUp_atNight/Positive/'        #For the work laptop

In [3]:
# transform_characteristics = transforms.Compose([transforms.Resize(255),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor()])
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset = datasets.ImageFolder(data_dir, transform=transform_characteristics)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

We use only the encoder part of the nertwork

In [7]:
class Autoencoder_latentSpace(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 1, 28, 28
        # 32, 3, 224, 224
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> N, 110, 44, 44
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # -> N, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # -> N, 450, 1, 1
        )
        
        # self.show_modules()
        self.show_weights()

    def forward(self, x):
        print("This is the forward function")
        encoded = self.encoder(x)
        return encoded
    
    def show_modules(self):
        print("This is the new function")
        # print(self.modules())
        i = 0
        for m in self.modules():
            print(m)
            print("i is: ", i)
            print("print the next module")
            i = i +1
    
    def show_weights(self):
        print("These are the weights function")
        # print(self.modules())
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == 0:
                    print("i is: ", i)
                    print("The type of the module weight is: ", type(m.weight))
                    print("The shape of the module weight is: ", m.weight.shape)
                    print("print the next module")
                    i = i +1
    def show_one_module_weights(self):
        print("Showing only one module's weight")
        # print(self.modules())
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == 0:
                    print("i is: ", i)
                    print("The weights are: ", m.weight)
                    i = i +1
 

In [8]:
model_encoder = Autoencoder_latentSpace()


These are the weights function
i is:  0
The type of the module weight is:  <class 'torch.nn.parameter.Parameter'>
The shape of the module weight is:  torch.Size([110, 3, 9, 9])
print the next module


In [9]:
model_encoder.show_one_module_weights()

Showing only one module's weight
i is:  0
The weights are:  Parameter containing:
tensor([[[[ 0.0622, -0.0594, -0.0225,  ..., -0.0568, -0.0121, -0.0302],
          [-0.0198, -0.0178, -0.0621,  ...,  0.0560, -0.0408,  0.0506],
          [ 0.0222,  0.0010,  0.0340,  ..., -0.0568, -0.0121,  0.0166],
          ...,
          [ 0.0062,  0.0475,  0.0581,  ..., -0.0124,  0.0480, -0.0438],
          [ 0.0528, -0.0314, -0.0181,  ...,  0.0492, -0.0063,  0.0520],
          [-0.0592,  0.0236, -0.0081,  ..., -0.0475, -0.0356,  0.0078]],

         [[-0.0489, -0.0399,  0.0422,  ...,  0.0099, -0.0005, -0.0627],
          [ 0.0360, -0.0232,  0.0040,  ...,  0.0130,  0.0102, -0.0291],
          [ 0.0448,  0.0430, -0.0187,  ...,  0.0131,  0.0595, -0.0061],
          ...,
          [ 0.0050,  0.0629, -0.0043,  ..., -0.0433, -0.0423, -0.0220],
          [ 0.0393,  0.0171,  0.0366,  ...,  0.0045, -0.0050,  0.0503],
          [-0.0170,  0.0313,  0.0293,  ..., -0.0524, -0.0154, -0.0146]],

         [[ 0.0045, 

In [11]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 1, 28, 28
        # 32, 3, 224, 224
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        
        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> N, 110, 44, 44
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # -> N, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # -> N, 450, 1, 1
        )
        
        # N , 450, 1, 1
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(output_channels_layer3, output_channels_layer2, 8),  # -> 32, 225, 8, 8
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels_layer2, output_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), 
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels, input_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), # N, 3, 224, 224
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
 
# Note: nn.MaxPool2d -> use nn.MaxUnpool2d, or use different kernelsize, stride etc to compensate...
# Input [-1, +1] -> use nn.Tanh

In [12]:
model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

In [13]:
filepath = "../../../BottlesAnomalies_TFM/models/pytorchModels/PytorchModel2"
# For loading the model 
model.load_state_dict(torch.load(filepath))
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

In [56]:
index = 0
i = 0
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        if i == index:
            print("i is: ", i)
            print("The weights are: ", m.weight)
        i = i +1

i is:  0
The weights are:  Parameter containing:
tensor([[[[-6.7870e-03, -5.9977e-03,  5.9349e-04,  ..., -3.1963e-03,
           -3.9213e-03, -3.6567e-03],
          [-6.5028e-03, -5.3889e-03, -5.1010e-03,  ..., -4.3888e-03,
           -2.7821e-02, -3.3790e-02],
          [-1.4501e-02, -2.2383e-02, -4.9993e-03,  ..., -4.7834e-03,
            5.3126e-03, -2.4254e-02],
          ...,
          [-7.1905e-03, -4.8892e-03, -4.2067e-03,  ..., -3.7277e-03,
           -3.2662e-02, -4.8444e-03],
          [ 1.3601e-02, -2.4451e-03,  1.2937e-02,  ..., -2.2916e-02,
           -3.0448e-04, -7.7624e-03],
          [-5.9227e-03,  1.0600e-02, -3.6416e-02,  ..., -5.6808e-03,
           -3.6151e-02, -1.7110e-02]],

         [[-2.6367e-02, -5.7756e-03, -2.7488e-02,  ...,  1.3288e-02,
           -4.9012e-03, -8.6235e-03],
          [-5.6408e-03, -6.7863e-03, -1.7314e-02,  ..., -1.7055e-02,
           -1.4709e-02, -4.5184e-03],
          [-3.1287e-03, -7.1006e-03, -4.9694e-03,  ..., -4.3487e-03,
         

In [29]:
layers_weights_list = []
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        weights = m.weight
        layers_weights_list.append(weights)
print("The shape of the weights list is: ", len(layers_weights_list))

The shape of the weights list is:  3


Copying the module's weight from one model to another. 

In [38]:
class Autoencoder_latentSpace(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 1, 28, 28
        # 32, 3, 224, 224
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        
        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> N, 110, 44, 44
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # -> N, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # -> N, 450, 1, 1
        )

    def forward(self, x):
        print("This is the forward function")
        encoded = self.encoder(x)
        return encoded
    
    def show_modules(self):
        print("This is the show modules function")
        i = 0
        for m in self.modules():
            print(m)
            print("i is: ", i)
            print("print the next module")
            i = i +1
            
    def show_one_layer_weights(self, index):
        print("This is the one layer show function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == index:
                    print("i is: ", i)
                    print("The weights are: ", m.weight)
                i = i +1
    
    def update_weights(self):
        print("updating weights function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight = layers_weights_list[i]
                i = i +1

                                        
 

In [39]:
model_encoder = Autoencoder_latentSpace()
# model.eval()

In [40]:
model_encoder.show_one_layer_weights(0)

This is the one layer show function
i is:  0
The weights are:  Parameter containing:
tensor([[[[-0.0480, -0.0020, -0.0296,  ..., -0.0028,  0.0505,  0.0363],
          [-0.0377,  0.0043,  0.0170,  ...,  0.0356, -0.0326,  0.0614],
          [-0.0239,  0.0364, -0.0340,  ..., -0.0559, -0.0330,  0.0513],
          ...,
          [ 0.0257,  0.0473,  0.0466,  ..., -0.0539, -0.0223,  0.0163],
          [-0.0363,  0.0441, -0.0331,  ..., -0.0140,  0.0133, -0.0042],
          [ 0.0107,  0.0330,  0.0584,  ..., -0.0161,  0.0494, -0.0171]],

         [[-0.0425, -0.0601,  0.0085,  ...,  0.0589,  0.0383,  0.0473],
          [ 0.0341, -0.0535, -0.0478,  ..., -0.0018,  0.0189, -0.0201],
          [ 0.0569, -0.0590,  0.0413,  ...,  0.0211,  0.0590,  0.0193],
          ...,
          [ 0.0506, -0.0289, -0.0388,  ...,  0.0231,  0.0635, -0.0567],
          [ 0.0168, -0.0326, -0.0053,  ...,  0.0105,  0.0042,  0.0521],
          [-0.0414,  0.0059,  0.0414,  ...,  0.0617,  0.0579, -0.0523]],

         [[ 0.047

In [41]:
model_encoder.update_weights()

updating weights function


In [55]:
model_encoder.show_one_layer_weights(0)

This is the one layer show function
i is:  0
The weights are:  Parameter containing:
tensor([[[[-6.7870e-03, -5.9977e-03,  5.9349e-04,  ..., -3.1963e-03,
           -3.9213e-03, -3.6567e-03],
          [-6.5028e-03, -5.3889e-03, -5.1010e-03,  ..., -4.3888e-03,
           -2.7821e-02, -3.3790e-02],
          [-1.4501e-02, -2.2383e-02, -4.9993e-03,  ..., -4.7834e-03,
            5.3126e-03, -2.4254e-02],
          ...,
          [-7.1905e-03, -4.8892e-03, -4.2067e-03,  ..., -3.7277e-03,
           -3.2662e-02, -4.8444e-03],
          [ 1.3601e-02, -2.4451e-03,  1.2937e-02,  ..., -2.2916e-02,
           -3.0448e-04, -7.7624e-03],
          [-5.9227e-03,  1.0600e-02, -3.6416e-02,  ..., -5.6808e-03,
           -3.6151e-02, -1.7110e-02]],

         [[-2.6367e-02, -5.7756e-03, -2.7488e-02,  ...,  1.3288e-02,
           -4.9012e-03, -8.6235e-03],
          [-5.6408e-03, -6.7863e-03, -1.7314e-02,  ..., -1.7055e-02,
           -1.4709e-02, -4.5184e-03],
          [-3.1287e-03, -7.1006e-03, -4.96

In [60]:
########################################################
# Calculate KDE using sklearn
from sklearn.neighbors import KernelDensity

#Get encoded output of input images = Latent space
# encoded_images = model_encoder(images)
encoded_images = []

for i in range(len(dataset)):
    X = dataset[i]
    image_in_tensor = X[0]
    with torch.no_grad():
        Y = model(image_in_tensor)  # should be same as X

    prediction = computePred(err)
    y_pred.append(prediction) # Save Prediction
    y_true.append(ground_truth) # Save Truth
print(y_true)
print(y_pred)

This is the forward function


In [ ]:
print(encoded_images.shape)